# Introduction

Objective: Competition's objective is to predict short-term volatility for 112 stocks. In this notebook, I'll attempt to predict short-term volatility for only one stock, and then do the same for the rest of the 111 stocks. Let's figure out how to do so. 

# Optiver Realized Volatility Prediction

In [ ]:
import os
import pandas as pd
import numpy as np

##### Weighted Average Price (WAP)
###### Weighted Average Price (WAP) considers both the best bid price, the best ask price, and their respective volumes (bid size and ask size)

In [ ]:
def calculate_WAP(BidPrice_1, BidSize_1, AskPrice_1, AskSize_1):
    return (BidPrice_1 * AskSize_1 + BidSize_1 * AskPrice_1) / (BidSize_1 + AskSize_1)

##### Log Returns

In [ ]:
def calculate_log_returns(list_stock_prices):
    return np.log(list_stock_prices).diff()

##### Realized Volatility Calculation Formula

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

##### WAP

In [ ]:
calculate_WAP(147, 251, 148, 221)

147.53177966101694

##### Test Competition Data

In [ ]:
train = pd.read_csv('train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


##### Reading Book and Trade Test Parquet Files

In [ ]:
book = pd.read_parquet('book_test.parquet/stock_id=0/7832c05caae3489cbcbbb9b02cf61711.parquet')
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15


In [ ]:
trade = pd.read_parquet('trade_test.parquet/stock_id=0/31c83a67d81349208e7d5eace9dbbac8.parquet')
trade.head()

,time_id,seconds_in_bucket,price,size,order_count
0,4,7,1.000344,1,1
1,4,24,1.000049,100,7
2,4,27,1.000059,100,3


In [ ]:
book['wap'] = calculate_WAP(book['bid_price1'], book['bid_size1'], book['ask_price1'], book['ask_size1'])
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24,1.000307
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20,1.000307
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15,1.000601


##### Naive Prediction

In [ ]:
def realized_volatility_per_time_id(df_book_data):
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(calculate_log_returns)

In [ ]:
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24,1.000307
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20,1.000307
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15,1.000601


In [ ]:
pd.read_parquet('book_train.parquet/stock_id=0')

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100
...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28


In [ ]:
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24,1.000307
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20,1.000307
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15,1.000601


In [ ]:
# Add a stock id column in each book data parquet file
for i in range(0, 127):
    if os.path.exists(f'book_train.parquet/stock_id={i}'):
        book = pd.read_parquet(f'book_train.parquet/stock_id={i}')
        book['stock_id'] = i

# Optiver Realized Volatility Prediction

In [ ]:
import os
import pandas as pd
import numpy as np

##### Weighted Average Price (WAP)
###### Weighted Average Price (WAP) considers both the best bid price, the best ask price, and their respective volumes (bid size and ask size)

In [ ]:
def calculate_WAP(BidPrice_1, BidSize_1, AskPrice_1, AskSize_1):
    return (BidPrice_1 * AskSize_1 + BidSize_1 * AskPrice_1) / (BidSize_1 + AskSize_1)

##### Log Returns

In [ ]:
def calculate_log_returns(list_stock_prices):
    return np.log(list_stock_prices).diff()

##### Realized Volatility Calculation Formula

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

##### WAP

In [ ]:
calculate_WAP(147, 251, 148, 221)

147.53177966101694

##### Test Competition Data

In [ ]:
train = pd.read_csv('train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


##### Reading Book and Trade Test Parquet Files

In [ ]:
book = pd.read_parquet('book_test.parquet/stock_id=0/7832c05caae3489cbcbbb9b02cf61711.parquet')
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15


In [ ]:
trade = pd.read_parquet('trade_test.parquet/stock_id=0/31c83a67d81349208e7d5eace9dbbac8.parquet')
trade.head()

,time_id,seconds_in_bucket,price,size,order_count
0,4,7,1.000344,1,1
1,4,24,1.000049,100,7
2,4,27,1.000059,100,3


In [ ]:
book['wap'] = calculate_WAP(book['bid_price1'], book['bid_size1'], book['ask_price1'], book['ask_size1'])
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24,1.000307
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20,1.000307
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15,1.000601


##### Naive Prediction

In [ ]:
def realized_volatility_per_time_id(df_book_data):
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(calculate_log_returns)

In [ ]:
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24,1.000307
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20,1.000307
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15,1.000601


In [ ]:
pd.read_parquet('book_train.parquet/stock_id=0')

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100
...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28


In [ ]:
book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24,1.000307
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20,1.000307
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15,1.000601


In [ ]:
# Add a stock id column in each book data parquet file
# import glob
# for i in range(0, 127):
#     folder_path = f'book_train.parquet/stock_id={i}/'
#     parquet_files = glob.glob(f'{folder_path}*.parquet')
#     for file_path in parquet_files:
#         book = pd.read_parquet(file_path)
#         book.insert(0, 'stock_id', i)
#         book.to_parquet(file_path, index=False)

In [ ]:
book = pd.read_parquet('book_train.parquet/stock_id=124')
book.head()

,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,124,5,0,1.003142,1.003260,1.003023,1.003379,400,150,640,301
1,124,5,1,1.003379,1.003616,1.003260,1.003735,235,435,800,760
2,124,5,2,1.003260,1.003497,1.003142,1.003616,235,357,400,500
3,124,5,3,1.003379,1.003497,1.003260,1.003616,200,212,456,300
4,124,5,4,1.003260,1.003379,1.003142,1.003497,457,100,156,202


In [ ]:
# Add a stock id column in each trade data parquet file
# import glob
# for i in range(0, 127):
#     folder_path = f'trade_train.parquet/stock_id={i}/'
#     parquet_files = glob.glob(f'{folder_path}*.parquet')
#     for file_path in parquet_files:
#         trade = pd.read_parquet(file_path)
#         trade.insert(0, 'stock_id', i)
#         trade.to_parquet(file_path, index=False)

In [ ]:
trade = pd.read_parquet('trade_train.parquet/stock_id=124')
trade.head()

,stock_id,time_id,seconds_in_bucket,price,size,order_count
0,124,5,0,1.003142,111,3
1,124,5,1,1.003577,9502,65
2,124,5,3,1.003269,867,12
3,124,5,5,1.003716,717,9
4,124,5,6,1.003470,641,8


##### Handled cleaning of data by adding first column as stock_id

In [ ]:
def realized_volatility_per_time_id(df_book_data):
    df_book_data['log_return'] = df_book_data.groupby(['stock_id','time_id'])['wap'].transform(calculate_log_returns)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock = pd.DataFrame(df_book_data.groupby(['stock_id','time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':'prediction'})
    return df_realized_vol_per_stock

#### Add a 'WAP' calculated column in each dataframes; test it with one first

In [ ]:
book = pd.read_parquet('book_train.parquet/stock_id=124')
book['wap'] = calculate_WAP(book['bid_price1'], book['bid_size1'], book['ask_price1'], book['ask_size1'])
df_realized_train = realized_volatility_per_time_id(book)


In [ ]:
train = pd.read_csv('train.csv')
df_joined = train.merge(df_realized_train, on=['stock_id', 'time_id'], how='left')
df_joined.head()

,stock_id,time_id,target,prediction
0,0,5,0.004136,NaN
1,0,11,0.001445,NaN
2,0,16,0.002168,NaN
3,0,31,0.002195,NaN
4,0,62,0.001747,NaN
